In [1]:
"""
import packages
"""
import os, glob
import json
import pandas as pd
import numpy as np
import math, random
from datetime import datetime
import pickle
from matplotlib import pyplot as plt
import argparse
import path, sys, re, time
from collections import Counter
from scipy.spatial import distance_matrix
from scipy.signal import find_peaks

In [2]:
"""
import custom packages
"""
from module_.info.testbed_info import d_files, g_files, c_files, p_files
from module_.info.config import config, feature_name
from module_.readText import create_episodes, time_correction
from module_.featureExtraction import feature_extraction
from module_.changePointDetection import change_point_detection
# from module_.evaluation import evaluation_
# from module_.analysis import neighbor_events as ne
# from module_.helper.labeling import feature_label

# Testbed (Seminar, multi-resident, episodes)

In [5]:
"""
load raw files
"""
dir_="dataset/testbed/discrete/"
task_dict={
    0:  [np.load(dir_+file_name) for file_name in g_files.keys()],
    1:  [np.load(dir_+file_name) for file_name in d_files.keys()],
    2:  [np.load(dir_+file_name) for file_name in c_files.keys()],
    3:  [np.load(dir_+file_name) for file_name in p_files.keys()]
}
name_dict={
    0: 'G', 1: 'D', 2: 'C', 3: 'P'
}

episodes, trs, tags = create_episodes(task_dict, name_dict)
episodes=[time_correction(eps, trs[i]) for i, eps in enumerate(episodes)]

In [ ]:
"""make combinations
    1. pick one group type
    2. pick an activity stream from the group
    3. pick another group type w/o type 1.
    4. pick an activity stream from the group
"""
data_name='testbed'
metric='SEP'

for i, eps in enumerate(episodes):
    pairname=tags[i][0]+tags[i][2]
    print(i, tags[i])
    sensor_list=sorted(set(eps[:,0]))
    features=feature_extraction(eps, data_name, sensor_list)
    scores=change_point_detection(features, i, pairname, data_name=data_name, metric=metric, save=True)
    scores=np.array(scores)
    scores[scores<0]=0

    peak, _ =find_peaks(scores)
    positive=[i for i in peak if scores[i]>0.3]
    # positive=[i for i in range(len(scores)) if scores[i]>0.45]
    
###
    plt.title("{}-{}".format(tags[i], i))
    plt.ylabel('score')
    plt.xlabel('event')
    plt.ylim(0,0.7)
    plt.bar(range(len(eps)), scores)
    plt.plot(positive, np.array(scores)[positive], 'bx', label='peak')
    plt.axhline(y=0.45, linestyle=':', color='r', label='threshold')
    plt.axvline(x=trs[i], linestyle=':', color='g', label='transition')
    plt.legend()
    plt.savefig("./outputs/{}/{}/{}/{}/graph.png".format(data_name, metric, pairname, i))
    plt.clf()

###

# Evaluation

In [ ]:
"""
    hh101 Evaluation
    - load scores
"""
total_counts=np.zeros(4)
denom = numer = 0
for activity_folder in glob.glob("./outputs/{}/{}/*".format(data_name, metric)):
    # one type of pairs
    activity_pair=activity_folder.split("/")[-1]
    print(activity_pair)
    pair_counts=np.zeros(4) # TP, FP, TN, FN

    for episode_folder in glob.glob("{}/*".format(activity_folder)):
        denom+=1
        eps_order=int(episode_folder.split("/")[-1])
        eps, point=episodes[eps_order], trs[eps_order]
        scores=np.load("{}/scores.npy".format(episode_folder))

        peaks, _ = find_peaks(scores)
        positives=[i for i in peaks if scores[i]>0.1]
        numer+=len(positives)
        # positives=[i for i in range(len(scores)) if scores[i]>0.3]
        ttimestamp=float(eps[point][2])

        for i in range(len(scores)):
            if i in positives:
                if i==point:
                    pair_counts[0]+=1
                else:
                    timestamp_b=float(eps[i-1][2])
                    timestamp_a=float(eps[i][2])
                    if abs(ttimestamp-timestamp_b)<30 or abs(ttimestamp-timestamp_a)<30:
                        pair_counts[0]+=1
                    else:
                        pair_counts[1]+=1
            else:
                if i==point:
                    pair_counts[3]+=1
                else:
                    pair_counts[2]+=1
    TPR_=pair_counts[0]/(pair_counts[0]+pair_counts[3])
    FPR_=pair_counts[1]/(pair_counts[1]+pair_counts[2])
    print("Avg. TPR and FPR: ({}, {})".format(TPR_, FPR_))

    total_counts+=pair_counts

TPR=total_counts[0]/(total_counts[0]+total_counts[3])
FPR=total_counts[1]/(total_counts[1]+total_counts[2])
print("Total Avg. TPR and FPR: ({}, {})".format(TPR, FPR))

print(numer/denom)

# Positive Analysis

In [8]:
data_name='testbed'
metric='SEP'
for activity_folder in glob.glob("./outputs/{}/{}/*".format(data_name, metric)):
    activity_pair=activity_folder.split("/")[-1]
    print(activity_pair)

    for episode_folder in glob.glob("{}/*".format(activity_folder)):
        eps_order=int(episode_folder.split("/")[-1])
        eps, point=episodes[eps_order], trs[eps_order]
        sensor_list=sorted(set(eps[:,0]))
        features=feature_extraction(eps, data_name, sensor_list)
        features=np.array(features)
        scores=np.load("{}/scores.npy".format(episode_folder))
        lambdas=np.load("{}/lambdas.npy".format(episode_folder))
        sigmas=np.load("{}/sigmas.npy".format(episode_folder))
        thetas=np.load("{}/thetas.npy".format(episode_folder))

        peaks, _ = find_peaks(scores)
        positives=[p for p in peaks if scores[p]>0.1]
        
        ttimestamp=float(eps[point][2])
        true_positives=[p for p in positives if abs(ttimestamp-float(eps[p][2]))<=30]
        false_positives=[p for p in positives if p not in true_positives]
        
        # creating window of the latest event
        events_true_positives=[]
        for p in true_positives:
            low_bound=p-30+1
            if low_bound<0:
                repeat=np.array([0 for _ in range(30-p-1)])
                window=np.concatenate((repeat, eps[:p+1]), axis=0)
            else:
                window=eps[low_bound:p+1,:]
            window=np.concatenate((window, np.array([eps[min(len(eps)-1,p+1)], eps[min(len(eps)-1,p+2)]])), axis=0)
            events_true_positives.append(window)
        
        events_false_positives=[]
        for p in false_positives:
            low_bound=p-30+1
            if low_bound<0:
                repeat=np.array([eps[0] for _ in range(30-p-1)])
                window=np.concatenate((repeat, eps[:p+1]), axis=0)
            else:
                window=eps[low_bound:p+1,:]
            window=np.concatenate((window, np.array([eps[min(len(eps)-1,p+1)], eps[min(len(eps)-1,p+2)]])), axis=0)
            events_false_positives.append(window)

        # events_true_positives=[eps[max(0, p-30):min(len(eps)-1, p+3)] for p in true_positives]
        # events_false_positives=[eps[max(0, p-30):min(len(eps)-1, p+3)] for p in false_positives]

        features_true_positives=[np.array(features)[[max(0,p-1),p,min(len(features)-1,p+1),min(len(features)-1,p+2)]] for p in true_positives]
        features_false_positives=[np.array(features)[[max(0,p-1),p,min(len(features)-1,p+1),min(len(features)-1,p+2)]] for p in false_positives]

        parameters_true_positives=[[lambdas[p], sigmas[p][0], sigmas[p][1]] for p in true_positives]
        parameters_false_positives=[[lambdas[p], sigmas[p][0], sigmas[p][1]] for p in false_positives]

        positive_folder="{}/positive".format(episode_folder)
        true_folder="{}/true".format(positive_folder)
        false_folder="{}/false".format(positive_folder)

        if not os.path.exists(positive_folder):
            os.mkdir(positive_folder)
        if not os.path.exists(true_folder):
            os.mkdir(true_folder)
        if not os.path.exists(false_folder):
            os.mkdir(false_folder)

        np.save("{}/events.npy".format(true_folder), events_true_positives)
        np.save("{}/features.npy".format(true_folder), features_true_positives)
        np.save("{}/parameters.npy".format(true_folder), parameters_true_positives)

        np.save("{}/events.npy".format(false_folder), events_false_positives)
        np.save("{}/features.npy".format(false_folder), features_false_positives)
        np.save("{}/parameters.npy".format(false_folder), parameters_false_positives)
        

GC
GP
PG
PD
DG
CD
CG
PC
DC
CP
DP
GD
